# Advanced Querying Mongo

Importing libraries and setting up connection

In [6]:
import pymongo
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
c = db.get_collection("companies")

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [14]:
# Your Code
proy={'name':1,'_id': 0}
list(c.find({"name": {"$in": ['Babelgum']}}, proy))


[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [16]:
# Your Code
list(c.find({'number_of_employees': {"$gt": 5000}}, proy).limit(10))

[{'name': 'Facebook'},
 {'name': 'eBay'},
 {'name': 'Cisco'},
 {'name': 'Yahoo!'},
 {'name': 'Google'},
 {'name': 'Intel'},
 {'name': 'Nintendo'},
 {'name': 'Adobe Systems'},
 {'name': 'Sony'},
 {'name': 'PayPal'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [22]:
# Your Code
proy2={'name':1,'founded_year':1,'_id': 0}
cond1={'founded_year':{"$gte":2000}}
cond2={'founded_year':{"$lte":2005}}
     
list(c.find({"$and": [cond1,cond2]},proy2).limit(10))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [27]:
# Your Code
proy3={'name':1,'ipo':1,'_id': 0}
cond3={'ipo.valuation_amount':{"$gt":100000000}}
cond4={'founded_year':{"$lte":2010}}
list(c.find({"$and": [cond3,cond4]},proy2).limit(10))

[{'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Twitter', 'founded_year': 2006},
 {'name': 'Yelp', 'founded_year': 2004},
 {'name': 'LinkedIn', 'founded_year': 2003},
 {'name': 'Amazon', 'founded_year': 1994},
 {'name': 'Brightcove', 'founded_year': 2004},
 {'name': 'KIT digital', 'founded_year': 2008},
 {'name': 'Nielsen', 'founded_year': 1922},
 {'name': 'OpenTable', 'founded_year': 1998},
 {'name': 'ChannelAdvisor', 'founded_year': 2001}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [33]:
# Your Code
cond5={'number_of_employees': {"$lt": 1000}}
cond6={'founded_year':{"$lte":2005}}

list(c.find({"$and": [cond5,cond6]},proy).limit(10).sort('number_of_employees',-1))


[{'name': 'Infinera Corporation'},
 {'name': 'Box'},
 {'name': 'NorthPoint Communications Group'},
 {'name': '888 Holdings'},
 {'name': 'Forrester Research'},
 {'name': 'Webmetrics'},
 {'name': 'SonicWALL'},
 {'name': 'Workday'},
 {'name': 'Cornerstone OnDemand'},
 {'name': 'Mozilla'}]

### 6. All the companies that don't include the `partners` field.

In [40]:
# Your Code
proy4={"name":1,"partners":1,"_id":0}
list(c.find({"partners":{"$exists": False}},proy4).limit(10))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [46]:
# Your Code
list(c.find({"category_code": {"$in":["null"]}},proy).limit(10))

[]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [47]:
# Your Code
proy5={'name':1,'number_of_employees':1,'_id': 0}
cond7={'number_of_employees':{"$gte":100}}
cond8={'number_of_employees':{"$lt":1000}}
     
list(c.find({"$and": [cond7,cond8]},proy5).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [55]:
# Your Code
proy6={'name':1,'ipo.valuation_amount':1,'_id': 0}    
list(c.find({},proy6).limit(10).sort('ipo.valuation_amount',-1))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [56]:
# Your Code
proy7={'name':1,'number_of_employees':1,'_id': 0}    

list(c.find({},proy7).limit(10).sort('number_of_employees',-1))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [79]:
# Your Code
list(c.find({'founded_month':{"$gte":7}},proy).limit(10).sort('founded_month',-1))
#para tener 1000 

[{'name': 'Kyte'},
 {'name': 'Swivel'},
 {'name': 'FlickIM'},
 {'name': 'Ikan'},
 {'name': 'Wesabe'},
 {'name': 'Sway'},
 {'name': 'hi5'},
 {'name': 'Socialtext'},
 {'name': 'blinkx'},
 {'name': 'Thoof'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [65]:
# Your Code
cond10={'acquisition.price_amount': {"$gt": 10000000}}
cond11={'founded_year':{"$lt":2000}}

list(c.find({"$and": [cond10,cond11]},proy).limit(10).sort('acquisition.price_amount',-1))




[{'name': 'BEA Systems'},
 {'name': 'Navteq'},
 {'name': 'Sun Microsystems'},
 {'name': 'Pixar'},
 {'name': 'LSI'},
 {'name': 'National Semiconductor'},
 {'name': 'aQuantive'},
 {'name': 'Siebel Systems'},
 {'name': 'Sybase'},
 {'name': 'Affiliated Computer Services'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [70]:
# Your Code
proy8={'name':1,'acquisition':1,'_id': 0}   

list(c.find({'acquisition.acquired_year':{"$gt":2010}},proy8).limit(1).sort('acquisition.price_amount',-1))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [74]:
list(c.find({},proy2).limit(10).sort('founded_year',-1))

[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [75]:
# Your Code
list(c.find({'founded_day':{"$lte":7}},proy).limit(10).sort('acquisition.price_amount',-1))


[{'name': 'Netscape'},
 {'name': 'PayPal'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Postini'},
 {'name': 'Danger'},
 {'name': 'Clearwell Systems'},
 {'name': 'PrimeSense'},
 {'name': 'Amobee'},
 {'name': 'BlueLithium'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [77]:
# Your Code
cond12={"category_code": {"$in":["web"]}}
cond13={'number_of_employees':{"$gt":4000}}


list(c.find({"$and": [cond12,cond13]},proy7).limit(10).sort('number_of_employees',1))


[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [78]:
# Your Code
cond14={"acquisition.price_currency_code": {"$in":["EUR"]}}
cond15={'acquisition.price_amount':{"$gt":10000000}}

list(c.find({"$and": [cond14,cond15]},proy6).limit(10).sort('acquisition.price_amount',-1))


[{'name': 'Apertio'},
 {'name': 'Webedia'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'},
 {'name': 'Greenfield Online'},
 {'name': 'ZYB'},
 {'name': 'Wayfinder'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [82]:
# Your Code
list(c.find({'founded_month':{"$lte":6}},proy8).limit(2))

[{'name': 'Facebook', 'acquisition': None},
 {'name': 'Postini',
  'acquisition': {'price_amount': 625000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://www.techcrunch.com/2007/07/09/google-acquires-postini-for-625-million/',
   'source_description': 'http://www.techcrunch.com/2007/07/09/google-acquires-postini-for-625-million/',
   'acquired_year': 2007,
   'acquired_month': 7,
   'acquired_day': 9,
   'acquiring_company': {'name': 'Google', 'permalink': 'google'}}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [87]:
# Your Code
cond16={'founded_year': {"$gte": 2000}}
cond17={'founded_year': {"$lte": 2010}}
cond18={'acquisition.acquired_year': {"$gte": 2011}}

list(c.find({"$and": [cond16,cond17,cond18]},proy).limit(10))

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'Kyte'},
 {'name': 'Jingle Networks'},
 {'name': 'blogTV'},
 {'name': 'delicious'},
 {'name': 'Revision3'},
 {'name': 'iContact'},
 {'name': 'Mashery'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code
